<a href="https://colab.research.google.com/github/balajisriraj/Twitter-Summary-mini-Project/blob/main/Twitter_Trend_Finder_In_Progress_V0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Flow:

- Connect to Twittter API using the required creds
- Find the Top trending Hashtags
- Then filter the hashtags which are in English language
- Filter Top 3 Hashtags from that
- For these 3 hashtags, fetch 100 Tweets individually
- Combain all the 100 tweets into one big corpus
- Do basic Data Cleaning & Processing for better Tokenization
- Using Text to Text transformer t5 model Summarize the tweets
- Show Summarized text output for each Hashtag

Import Python Modules

In [124]:
import pandas as pd
from tweepy import OAuthHandler
from tweepy import API
from googletrans import Translator
import GetOldTweets3 as got

In [115]:
#Past the keys from your local drive

In [3]:
# Consumer key authentication(consumer_key,consumer_secret can be collected from our twitter developer profile)
auth = OAuthHandler(consumer_key, consumer_secret)
# Access key authentication(access_token,access_token_secret can be collected from our twitter developer profile)
auth.set_access_token(access_token, access_token_secret)
# Set up the API with the authentication handler
api = API(auth)

In [85]:
WOE_ID = 2295424 # Where on Earth id can be extracted from https://nations24.com/world-wide
lan_find = Translator() # for finding teh language of the hashtags

In [113]:
def get_location_trends(locations, auth,n_hashtags,lang):
  api = API(auth)
  trends = api.trends_place(locations)
  data = trends[0]
  trends_data = data['trends']
  global tred_data
  tred_data = []
  for info in trends_data:
    tred_data.append([info['name'],info['tweet_volume'],lan_find.detect(info['name']).lang ] )
  tred_data = pd.DataFrame(tred_data, columns = list(['Hashtag',
                                                    'Tweet_Volume', 'Language'])).sort_values(by = ['Tweet_Volume'],ascending = False)
  #select english language trends
  tred_data = tred_data[tred_data.Language == lang]
  #select top 3 trends
  tred_data = tred_data.nlargest(n_hashtags,columns=['Tweet_Volume'])
  return tred_data

In [116]:
df_trending = get_location_trends(WOE_ID,auth,3,'en')

In [120]:
df_trending

,Hashtag,Tweet_Volume,Language
26,#PCAs,3125496.0,en
18,#SavageLoveRemix,1203056.0,en
5,#TrumpHasCovid,451034.0,en


In [118]:
search_words = ["#domesticviolence", "#domesticabuse", "#abuse"] 
SINCE = "2020-01-01"
UNTIL = "2020-05-20"
MAXTWEET = 400

In [125]:
def get_tweets(state, startdate, enddate, maxtweet, query, radius):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query)\
                                            .setSince(startdate)\
                                            .setUntil(enddate)\
                                            .setNear(state)\
                                            .setWithin(radius)\
                                            .setMaxTweets(maxtweet)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    
    text_tweets = [[tw.username,
                tw.text,
                tw.date,
                tw.retweets,
                tw.favorites,
                tw.mentions,
                tw.hashtags,
                tw.geo] for tw in tweet]
    df_state= pd.DataFrame(text_tweets, columns = ['User', 'Text', 'Date', 'Favorites', 'Retweets', 'Mentions','Hashtags', 'Geolocation'])
    df_state['search_query'] = query
    df_state['location'] = state
    return df_state

In [126]:
get_tweets(state='chennai',startdate=SINCE, enddate= UNTIL, maxtweet=100, query='#TrumpHasCovid', radius=100)

An error occured during an HTTP request: HTTP Error 404: Not Found
Try to open in browser: https://twitter.com/search?q=%23TrumpHasCovid%20near%3A%22chennai%22%20within%3A100%20since%3A2020-01-01%20until%3A2020-05-20&src=typd


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


,Hashtag,Tweet_Volume,Language
19,#SavageLoveRemix,1195543.0,en
11,#TrumpHasCovid,443265.0,en


In [91]:
lang = detect("#सुदर्शन_की_हार_हिंदुओ_की_हार")
print(lang)


hi
